# Data Analysis for CS506 Project

In CS506_DataProcessing_Graph.ipynb file, datasets are processed and cleaned finally store in 2 csv files. Two files are: **df_201x_BU_analysis_E_Final.csv** and **df_201x_BU_analysis_G_Final.csv**. In this analysis, we directly use those two files to do the analysis

In [1]:
# Import files
import csv
import pandas as pd
import numpy as np
from sklearn.preprocessing import normalize
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from statsmodels.formula.api import ols

import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
# import statsmodels.formula.api as smf

/Users/qianzhang/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
# Read data
df_BU_analysis_E = pd.read_csv('df_201x_BU_analysis_E_Final.csv', index_col=0)
df_BU_analysis_G = pd.read_csv('df_201x_BU_analysis_G_Final.csv', index_col=0)

# Data for E and G
print(df_BU_analysis_G.columns)
print(df_BU_analysis_E.columns)

Index(['Temperature', 'WI', 'GHGI', 'Age', 'Uses_College', 'Uses_Sports',
       'Uses_ResidentHousing', 'Uses_Laboratory', 'Uses_Office', 'Uses_Hotel',
       'Uses_Distribution', 'Uses_Food', 'Uses_Worship', 'Uses_Medical',
       'Y_Gas', 'Y_G_Level'],
      dtype='object')
Index(['Temperature', 'WI', 'GHGI', 'Age', 'Uses_College', 'Uses_Sports',
       'Uses_ResidentHousing', 'Uses_Laboratory', 'Uses_Office', 'Uses_Hotel',
       'Uses_Distribution', 'Uses_Food', 'Uses_Worship', 'Uses_Medical',
       'Y_Electric', 'Y_E_Level'],
      dtype='object')


In [3]:
# Browsing data for Electricity
# df_BU_analysis_E.head(10)

In [4]:
# Browsing data for Gas
# df_BU_analysis_G.head(10)

In [5]:
# Generating Training and Testing sets for 2 regressions (Electricity, Gas)
df_BU_analysis_E_shuffle = df_BU_analysis_E.sample(frac = 1)
df_BU_analysis_G_shuffle = df_BU_analysis_G.sample(frac = 1)

num_e = int(len(df_BU_analysis_E_shuffle)*0.9)
num_g = int(len(df_BU_analysis_G_shuffle)*0.9)

df_BU_training_E = df_BU_analysis_E_shuffle.iloc[:num_e, :]
df_BU_testing_E = df_BU_analysis_E_shuffle.iloc[num_e:, :]

df_BU_training_G = df_BU_analysis_G_shuffle.iloc[:num_g, :]
df_BU_testing_G = df_BU_analysis_G_shuffle.iloc[num_g:, :]

X_column = ['Temperature', 'WI', 'GHGI', 'Age', 'Uses_College', 'Uses_Sports', 'Uses_ResidentHousing', 'Uses_Laboratory', 'Uses_Office', 'Uses_Hotel', 'Uses_Distribution', 'Uses_Food', 'Uses_Worship', 'Uses_Medical']
# X_column = ['Temperature', 'WI', 'Age', 'Uses_College', 'Uses_Sports', 'Uses_ResidentHousing', 'Uses_Laboratory', 'Uses_Office', 'Uses_Hotel', 'Uses_Distribution', 'Uses_Food', 'Uses_Worship', 'Uses_Medical']


In [6]:
def printVariables():
    for i in range(len(X_column)):
        print('x{}: {}'.format(i+1, X_column[i]))
        

## Linear Regression

In [7]:
# Statesmodel Version of LR
# Traning Data

X_E = df_BU_training_E[X_column]
Y_E = df_BU_training_E['Y_Electric']

X_G = df_BU_training_G[X_column]
Y_G = df_BU_training_G['Y_Gas']


# Testing Data
X_E_test = df_BU_testing_E[X_column]
Y_E_test = df_BU_testing_E['Y_Electric']

X_G_test = df_BU_testing_G[X_column]
Y_G_test = df_BU_testing_G['Y_Gas']

print('BU_data shape')
print(df_BU_analysis_E.shape)
print(df_BU_analysis_G.shape)
print(df_BU_analysis_E_shuffle.shape)
print(df_BU_analysis_G_shuffle.shape)
print()
print('Training Data Shape:')
print(X_E.shape)
print(X_G.shape)
print()
print(Y_E.shape)
print(Y_G.shape)
print()
print('Testing Data Shape:')
print(X_E_test.shape)
print(X_G_test.shape)
print()
print(Y_E_test.shape)
print(Y_G_test.shape)


BU_data shape
(1512, 16)
(1404, 16)
(1512, 16)
(1404, 16)

Training Data Shape:
(1360, 14)
(1263, 14)

(1360,)
(1263,)

Testing Data Shape:
(152, 14)
(141, 14)

(152,)
(141,)


In [8]:
# Run the regression:
version = 'statsModel'
# version = 'sklearn'


if version == 'statsModel':
    # Electricity
    # OLS regression
    model_e = ols(
    formula = 'Y_Electric ~ Temperature + WI + GHGI + Age + Uses_College + Uses_Sports + Uses_ResidentHousing + Uses_Laboratory + Uses_Office + Uses_Hotel + Uses_Distribution + Uses_Food + Uses_Worship + Uses_Medical', 
    data = df_BU_training_E) 
    results_e = model_e.fit()
    
    # Gas
    # OLS regression 
    model_g = ols(
    formula = 'Y_Gas ~ Temperature + WI + GHGI + Age + Uses_College + Uses_Sports + Uses_ResidentHousing + Uses_Laboratory + Uses_Office + Uses_Hotel + Uses_Distribution + Uses_Food + Uses_Worship + Uses_Medical', 
    data = df_BU_training_G)

    results_g = model_g.fit()
    
else:
    
    # Using Sklearn linear model to do the prediction
    linearModel_E = LinearRegression(fit_intercept=True, normalize = True)
    linearModel_G = LinearRegression(fit_intercept=True, normalize = True)


    linearModel_E.fit(X_E, Y_E)
    linearModel_G.fit(X_G, Y_G)

    # linear regression accuracy through cross validation
    score_E = cross_val_score(linearModel_E, X_E, Y_E, cv=3)
    score_linear_mean_E = score_E.mean()
    score_G = cross_val_score(linearModel_G, X_G, Y_G, cv=3)
    score_linear_mean_G = score_G.mean()



    np.set_printoptions(suppress=True)

    print('E: Parameters')
    print(linearModel_E.coef_.shape)
    print(linearModel_E.intercept_)

    print()
    print('G: Parameters')
    print(linearModel_G.coef_.shape)
    print(linearModel_G.intercept_)


    coef_E = linearModel_E.coef_
    coef_G = linearModel_G.coef_

    coef_E = coef_E.reshape(coef_E.shape[0], 1)
    coef_G = coef_G.reshape(coef_G.shape[0], 1)

    df_coef_E = pd.DataFrame(coef_E.T, columns = X_column)
    df_coef_G = pd.DataFrame(coef_G.T, columns = X_column)

    df_coef_E['Const'] = linearModel_E.intercept_
    df_coef_G['Const'] = linearModel_G.intercept_


In [9]:
if version == 'sklearn':
    df_coef_E
else:
    printVariables()
    print()
    print(results_e.summary())

x1: Temperature
x2: WI
x3: GHGI
x4: Age
x5: Uses_College
x6: Uses_Sports
x7: Uses_ResidentHousing
x8: Uses_Laboratory
x9: Uses_Office
x10: Uses_Hotel
x11: Uses_Distribution
x12: Uses_Food
x13: Uses_Worship
x14: Uses_Medical

                            OLS Regression Results                            
Dep. Variable:             Y_Electric   R-squared:                       0.797
Model:                            OLS   Adj. R-squared:                  0.795
Method:                 Least Squares   F-statistic:                     377.9
Date:                Tue, 01 May 2018   Prob (F-statistic):               0.00
Time:                        21:17:57   Log-Likelihood:                -2567.3
No. Observations:                1360   AIC:                             5165.
Df Residuals:                    1345   BIC:                             5243.
Df Model:                          14                                         
Covariance Type:            nonrobust                           

In [10]:
if version == 'sklearn':
    df_coef_G
else:
    printVariables()
    print()
    print(results_g.summary())

x1: Temperature
x2: WI
x3: GHGI
x4: Age
x5: Uses_College
x6: Uses_Sports
x7: Uses_ResidentHousing
x8: Uses_Laboratory
x9: Uses_Office
x10: Uses_Hotel
x11: Uses_Distribution
x12: Uses_Food
x13: Uses_Worship
x14: Uses_Medical

                            OLS Regression Results                            
Dep. Variable:                  Y_Gas   R-squared:                       0.702
Model:                            OLS   Adj. R-squared:                  0.699
Method:                 Least Squares   F-statistic:                     226.6
Date:                Tue, 01 May 2018   Prob (F-statistic):          3.39e-317
Time:                        21:17:57   Log-Likelihood:                -3432.7
No. Observations:                1263   AIC:                             6893.
Df Residuals:                    1249   BIC:                             6965.
Df Model:                          13                                         
Covariance Type:            nonrobust                           

In [11]:
# # Computing MSE
# np.set_printoptions(suppress=True)

# #predicted comments according to wights
# def to_pred(f_coef,f_data):
#     # feature dot product with weights
#     # feature f_data from testing group
#     # weight f_coef from training
#     # predicted comments 
#     constant = f_coef.loc[0,'Const']
#     f_coef.drop(['Const'], axis = 1, inplace= True)
#     f_coef = f_coef.T
#     pred = np.dot(f_data,f_coef)
#     pred = pred + constant
#     return pred


# if version == 'sklearn':
#     #prediction of testing group 
#     y_pred_E = to_pred(df_coef_E,X_E_test)
#     y_pred_G = to_pred(df_coef_G,X_G_test)


#     #true value of testing group
#     y_true_E = Y_E_test
#     y_true_G = Y_G_test

#     #mean squared error
#     print('---------------Mean Suared Error of Electric---------------')
#     print(mean_squared_error(y_true_E, y_pred_E))
#     print('---------------Mean Suared Error of Gas---------------')
#     print(mean_squared_error(y_true_G, y_pred_G))
    
# else:
    
#     # Electricity
#     print('Electricity:')
#     const_e = results_e.params[0]
#     params_e = results_e.params[1:]
#     params_e = np.reshape(params_e, (params_e.shape[0], 1))

#     print('Constant:')
#     print(const_e)
#     print()
#     print('Parameters:')
#     print(params_e)
#     print() 

#     # Y_test
#     Y_hat_e = np.dot(x_eTest, params_e) + const_e
#     print('MSE:')
#     print(mean_squared_error(y_eTest, Y_hat_e))
    
#     print()
#     print('------------------------------')
#     print()
    
#     # Gas
#     print('Gas:')
#     const_g = results_g.params[0]
#     params_g = results_g.params[1:]
#     params_g = np.reshape(params_g, (params_g.shape[0], 1))

#     print('Constant:')
#     print(const_g)
#     print()
#     print('Parameters:')
#     print(params_g)
#     print() 

#     # Y_test
#     Y_hat_g = np.dot(x_gTest, params_g) + const_g
#     print('MSE:')
#     print(mean_squared_error(y_gTest, Y_hat_g))
       

## Cross Validation - Linear Regression Regularized (Fail, Performace worse)

In [12]:
# Cross validation score for statsmodel linear regression
# score = mse.mean()

def cross_validation(df, cv, label, regularize, a):
    #shuffle dataframe
    df = df.sample(frac = 1)
    lst_mse = []
    row_num = int(len(df))
    num = int(row_num / cv)
    n = 0
    while n < row_num:
        n_next = n + num
        # split data into training and testing 
        if n_next >= row_num:
            df_testing = df.iloc[n:, :]
            df_training = df.drop(df.index[[n,]],axis = 0)
        else:
            df_testing = df.iloc[n:n_next,:]
            df_training = df.drop(df.index[[n,n_next]], axis = 0)
        
        X_train = df_training.drop([label],axis = 1)
        Y_train = df_training[label]
        X_test = df_testing.drop([label],axis = 1)
        Y_test = df_testing[label]
        mod = sm.OLS(Y_train,X_train)
        res = mod.fit()
        if regularize == True:
            res_reg = mod.fit_regularized(alpha = a, L1_wt = 0, start_params = res.params)
            Y_pred = res_reg.predict(X_test)
        else:
            Y_pred = res.predict(X_test)
        mse = mean_squared_error(Y_test, Y_pred)
        lst_mse.append(mse)
        n = n_next
        m_mse = np.mean(lst_mse)
    return m_mse 


In [144]:
score_E = cross_validation(df_BU_analysis_E, 5, 'Y_Electric', False, 0)
score_G = cross_validation(df_BU_analysis_G, 5, 'Y_Gas', False, 0)
print('Score of linear model on electricity is ', score_E)
print('Score of linear model on gas is ', score_G)

print('-------------------------------------------')

score_E_reg = cross_validation(df_BU_analysis_E, 5, 'Y_Electric', True, 0)
score_G_reg = cross_validation(df_BU_analysis_G, 5, 'Y_Gas', True, 0)
print('Score of regularized linear model on electricity is ', score_E_reg)
print('Score of regularized linear model on gas is ', score_G_reg)

Score of linear model on electricity is  1.064678202439731
Score of linear model on gas is  3.7834564585043715
-------------------------------------------
Score of regularized linear model on electricity is  1.3216278639844976
Score of regularized linear model on gas is  4.769451405216438


## Non-Linear Model (Quadratic term)

In [143]:
print(df_BU_training_E.columns)
print() 
print(df_BU_training_G.columns)

Index(['Temperature', 'WI', 'GHGI', 'Age', 'Uses_College', 'Uses_Sports',
       'Uses_ResidentHousing', 'Uses_Laboratory', 'Uses_Office', 'Uses_Hotel',
       'Uses_Distribution', 'Uses_Food', 'Uses_Worship', 'Uses_Medical',
       'Y_Electric', 'Y_E_Level'],
      dtype='object')

Index(['Temperature', 'WI', 'GHGI', 'Age', 'Uses_College', 'Uses_Sports',
       'Uses_ResidentHousing', 'Uses_Laboratory', 'Uses_Office', 'Uses_Hotel',
       'Uses_Distribution', 'Uses_Food', 'Uses_Worship', 'Uses_Medical',
       'Y_Gas', 'Y_G_Level'],
      dtype='object')


In [15]:
# df_BU_training_E = df_BU_analysis_E_shuffle.iloc[:num_e, :]
# df_BU_testing_E = df_BU_analysis_E_shuffle.iloc[num_e:, :]

# df_BU_training_G = df_BU_analysis_G_shuffle.iloc[:num_g, :]
# df_BU_testing_G = df_BU_analysis_G_shuffle.iloc[num_g:, :]

# X_column = ['Temperature', 'WI', 'GHGI', 'Age', 'Uses_College', 'Uses_Sports', 'Uses_ResidentHousing', 'Uses_Laboratory', 'Uses_Office', 'Uses_Hotel', 'Uses_Distribution', 'Uses_Food', 'Uses_Worship', 'Uses_Medical']


# Electricity

# quadratic OLS regression
model_e_q = ols(
    formula = 'Y_Electric ~ np.power(Temperature, 2) + WI + np.power(GHGI, 2) + Age + Uses_College + Uses_Sports + Uses_ResidentHousing + Uses_Laboratory + Uses_Office + Uses_Hotel + Uses_Distribution + Uses_Food + Uses_Worship + Uses_Medical', 
    data = df_BU_training_E) 

results_e_q = model_e_q.fit()

# np.power(GHGI, 2)

# Gas

# quadratic OLS regression 

model_g_q = ols(
    formula = 'Y_Gas ~ np.power(Temperature, 2) + WI + np.power(GHGI, 2) + Age + Uses_College + Uses_Sports + Uses_ResidentHousing + Uses_Laboratory + Uses_Office + Uses_Hotel + Uses_Distribution + Uses_Food + Uses_Worship + Uses_Medical', 
    data = df_BU_training_G)

results_g_q = model_g_q.fit()



In [16]:
printVariables()
print()
print(results_e_q.summary())

x1: Temperature
x2: WI
x3: GHGI
x4: Age
x5: Uses_College
x6: Uses_Sports
x7: Uses_ResidentHousing
x8: Uses_Laboratory
x9: Uses_Office
x10: Uses_Hotel
x11: Uses_Distribution
x12: Uses_Food
x13: Uses_Worship
x14: Uses_Medical

                            OLS Regression Results                            
Dep. Variable:             Y_Electric   R-squared:                       0.768
Model:                            OLS   Adj. R-squared:                  0.765
Method:                 Least Squares   F-statistic:                     317.5
Date:                Tue, 01 May 2018   Prob (F-statistic):               0.00
Time:                        21:17:58   Log-Likelihood:                -2660.0
No. Observations:                1360   AIC:                             5350.
Df Residuals:                    1345   BIC:                             5428.
Df Model:                          14                                         
Covariance Type:            nonrobust                           

In [17]:
printVariables()
print()
print(results_g_q.summary())

x1: Temperature
x2: WI
x3: GHGI
x4: Age
x5: Uses_College
x6: Uses_Sports
x7: Uses_ResidentHousing
x8: Uses_Laboratory
x9: Uses_Office
x10: Uses_Hotel
x11: Uses_Distribution
x12: Uses_Food
x13: Uses_Worship
x14: Uses_Medical

                            OLS Regression Results                            
Dep. Variable:                  Y_Gas   R-squared:                       0.655
Model:                            OLS   Adj. R-squared:                  0.651
Method:                 Least Squares   F-statistic:                     182.2
Date:                Tue, 01 May 2018   Prob (F-statistic):          3.05e-277
Time:                        21:17:58   Log-Likelihood:                -3526.1
No. Observations:                1263   AIC:                             7080.
Df Residuals:                    1249   BIC:                             7152.
Df Model:                          13                                         
Covariance Type:            nonrobust                           

## Cross Validation - Quadratic Nonlinear Regression

In [43]:
# Cross validation score for quadratic linear regression
# score = mse.mean()

def cross_validation_q(df, cv, label, regularize, a):
    #shuffle dataframe
    df = df.sample(frac = 1)
    lst_mse = []
    row_num = int(len(df))
    num = int(row_num / cv)
    n = 0
    while n < row_num:
        n_next = n + num
        # split data into training and testing 
        if n_next >= row_num:
            df_testing = df.iloc[n:, :]
            df_training = df.drop(df.index[[n,]],axis = 0)
        else:
            df_testing = df.iloc[n:n_next,:]
            df_training = df.drop(df.index[[n,n_next]], axis = 0)
        X_test = df_testing.drop([label],axis = 1)
        Y_test = df_testing[label]
        mod = ols(formula = label +'~ Temperature + WI + np.power(GHGI, 2)+ Age + Uses_College + Uses_Sports + Uses_ResidentHousing + Uses_Laboratory + Uses_Office + Uses_Hotel + Uses_Distribution + Uses_Food + Uses_Worship + Uses_Medical', 
                        data = df_training) 
        res = mod.fit()
        if regularize == True:
            res_reg = mod.fit_regularized(alpha = a, L1_wt = 0, start_params = res.params)
            Y_pred = res_reg.predict(X_test)
        else:
            Y_pred = res.predict(X_test)
        mse = mean_squared_error(Y_test, Y_pred)
        lst_mse.append(mse)
        n = n_next
        m_mse = np.mean(lst_mse)
    return m_mse 


In [111]:
df_test_E = df_BU_analysis_E.copy()
df_test_G = df_BU_analysis_G.copy()

print(df_test_E.columns.values)
print(df_test_G.columns.values)

['Temperature' 'WI' 'GHGI' 'Age' 'Uses_College' 'Uses_Sports'
 'Uses_ResidentHousing' 'Uses_Laboratory' 'Uses_Office' 'Uses_Hotel'
 'Uses_Distribution' 'Uses_Food' 'Uses_Worship' 'Uses_Medical'
 'Y_Electric' 'Y_E_Level']
['Temperature' 'WI' 'GHGI' 'Age' 'Uses_College' 'Uses_Sports'
 'Uses_ResidentHousing' 'Uses_Laboratory' 'Uses_Office' 'Uses_Hotel'
 'Uses_Distribution' 'Uses_Food' 'Uses_Worship' 'Uses_Medical' 'Y_Gas'
 'Y_G_Level']


In [141]:
# Cross Validation

df_test_E['WI Quadratic'] = df_test_E['WI'] * df_test_E['WI']
df_test_G['WI Quadratic'] = df_test_G['WI'] * df_test_G['WI']
df_test_E['GHGI Quadratic'] = df_test_E['GHGI'] * df_test_E['GHGI']
df_test_G['GHGI Quadratic'] = df_test_G['GHGI'] * df_test_G['GHGI']


score_E_q = cross_validation(df_test_E, 5, 'Y_Electric', False, 0)
score_G_q = cross_validation(df_test_G, 5, 'Y_Gas', False, 0)
print('Score of quadraitc nonlinear model on electricity is ', score_E_q)
print('Score of quadratic nonlinear model on gas is ', score_G_q)

print('-------------------------------------------')

score_E_q_reg = cross_validation(df_test_E, 5, 'Y_Electric', True, 0)
score_G_q_reg = cross_validation(df_test_G, 5, 'Y_Gas', True, 0)
print('Score of regularized quadratic nonlinear model on electricity is ', score_E_q_reg)
print('Score of regularized quadratic nonlinear model on gas is ', score_G_q_reg)

Score of quadraitc nonlinear model on electricity is  0.97396961613041
Score of quadratic nonlinear model on gas is  3.5959920577686453
-------------------------------------------
Score of regularized quadratic nonlinear model on electricity is  0.9826042052357892
Score of regularized quadratic nonlinear model on gas is  4.180232140391554


## Multi-class Logistic Regression

In [20]:
# Electricity

# 1. Logistic Regression
# Y_Very_Low_E


#----------------------

# 2. Logistic Regression
# Y_Low_E

#----------------------

# 3. Logistic Regression
# Y_Medium_E


#----------------------

# 4. Logistic Regression
# Y_High_E


#----------------------

# 5. Logistic Regression
# Y_Very_High_E




In [21]:
# Gas

# 1. Logistic Regression
# Y_Very_Low_G


#----------------------

# 2. Logistic Regression
# Y_Low_G

#----------------------

# 3. Logistic Regression
# Y_Medium_G


#----------------------

# 4. Logistic Regression
# Y_High_G


#----------------------

# 5. Logistic Regression
# Y_Very_High_G

## Test Accuracy (Logistic)

In [23]:
# Test for Electricity

# Test for Gas